In [9]:
import numpy as np
import tensorflow as tf
import time
from simulator.core.simulator import Simulator
from simulator import Config
from simulator.environment.task_environment import TaskEnvironment
from simulator.logger import LogOutputConsolePrint, LogOutputTextFile, Logger
from simulator.world_builds.box_world import BoxWorld
from simulator.reporters import TransitionReporter
from simulator.processes.task_generator import TaskGenerator
from simulator.task_multiplexing import TaskMultiplexerSelectorDql
from simulator.task_multiplexing import TaskMultiplexerSelectorLocal
from simulator.task_multiplexing import TaskMultiplexerSelectorRandom
from simulator.task_multiplexing import TaskMultiplexerSelectorRemote
from simulator.task_multiplexing import TaskMultiplexerSelectorGreedy

## Set simulator behaviour

In [10]:
Logger.registerLogOutput(LogOutputConsolePrint())
Logger.registerLogOutput(LogOutputTextFile("log\log" + str(time.time()) + ".log"))
np.random.seed(Config.get('random_seed'))
tf.random.set_seed(Config.get('random_seed'))

generate task list: this is called here to make sure all selectors receive the same list of tuples

In [ ]:
TaskGenerator.retrieveList()

## Creating an instace of Box World

In [11]:

world = BoxWorld()
edgeNodes, mobileNodes = world.build()
edgeReward, mobileReward = world.defaultRewards()


## Creating the simulator

In [ ]:
simulator = Simulator()

## Creating the reporters

In [12]:
mobileReporter = TransitionReporter(simulator, "pickle") 
edgeReporter = TransitionReporter(simulator, "pickle") 

## Task multiplexing

In [13]:
dqlSelectorGenerator = lambda state: TaskMultiplexerSelectorDql(state, Config.get("dql_training_buffer_size"))
localSelectorGenerator = lambda state: TaskMultiplexerSelectorLocal()
remoteSelectorGenerator = lambda state: TaskMultiplexerSelectorRemote()
randomSelectorGenerator = lambda state: TaskMultiplexerSelectorRandom()
greedySelectorGenerator = lambda state: TaskMultiplexerSelectorGreedy(state)

## Prepare and Run the Simulation

### Both dql

In [ ]:
taskEnvironment = TaskEnvironment(edgeNodes, mobileNodes, 
                                  edgeSelectorGenerator= dqlSelectorGenerator, 
                                  mobileSelectorGenerator= dqlSelectorGenerator,
                                  edgeRewardFunction= edgeReward,
                                  mobileRewardFunction=mobileReward)
taskEnvironment.initialize(simulator, [mobileReporter.addTransition], [edgeReporter.addTransition])
simulator.run()

## Report delay

In [15]:
Logger.log("mobile delay: " + str(mobileReporter.averageDelay()), 0)
Logger.log("mobile power consumed: " + str(mobileReporter.averagePowerConsumed()), 0)

mobile delay: 3.024390262330601
mobile power consumed: 0.6531383489400012


In [ ]:
Logger.closeLogOutputs()
mobileReporter.pickle()